In [23]:
import numpy as np
import autograd.numpy as np_
from autograd import grad
import matplotlib.pyplot as plt

## Exercício 1
**Objetivo: entender e usar uma formulação matricial para a regressão linear**

Até o momento, a função que executamos para o problema de aproximação de dados:

$$
f(x) = \hat{y} = ax + b.
$$

Quando fazemos a regressão linear, estamos assumindo que existem várias medições $(x_i, y_i)$. Cada uma dessas medições é diferente do valor estimado, isto é:

$$
e_i = y_i - \hat{y}_i
$$

A média do quadrado dessa diferença pode ser calculada ao longo das $n$ medições que fizermos:

$$
\text{EQM} = \frac{1}{n} \sum_{i=1}^n ||e_i||^2.
$$

Isso é uma maneira absolutamente válida de pensar. Porém, vamos agora transportar essa formulação para usar matrizes, e em depois verificaremos como isso pode nos ajudar.

Veja, poderíamos escrever nossa função $f(x)$ assumindo que a entrada é um vetor de medidas $x_i$. Daí, ficaríamos com:

$$
\begin{array}{rl}
f(\boldsymbol{x}) &= a \boldsymbol{x} + b 
= \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{bmatrix} \begin{bmatrix} a \end{bmatrix} + b
=  \begin{bmatrix} x_1 & 1 \\ x_2 & 1 \\ \vdots & \vdots \\ x_n & 1  \end{bmatrix} \begin{bmatrix} a \\ b \end{bmatrix} 
= \begin{bmatrix} \hat{y}_1 \\ \hat{y}_2 \\ \vdots \\ \hat{y}_n \end{bmatrix}
\end{array}
$$

Nessa formulaçao, a diferença entre as predições e os valores medidos pode ser expressa em um vetor:

$$
e = y - \hat{y} = \begin{bmatrix} y_1 - \hat{y}_1 \\ y_1 - \hat{y}_1 \\ \vdots \\ y_n - \hat{y}_n \end{bmatrix} = \begin{bmatrix} e_1 \\ e_2 \\ \vdots \\ e_n \end{bmatrix}
$$

**Exercício**:

Mostre que o EQM pode ser calculado como:

$$
\text{EQM} = \frac{1}{n} e^T e
$$ 

usando a formulação mostrada neste enunciado, e como o resultado é estritamente igual ao EQM calculado usando a notação somatória acima. Para isso, expanda o vetor $e$ e mostre explicitamente como a operação resultante é igual à que obtivemos anteriormente.



# Exercício 2
**Objetivo: usar matrizes para representar modelos com várias entradas**

Até o momento, usamos a formulação:

$$
 \begin{bmatrix} \hat{y}_1 \\ \hat{y}_2 \\ \vdots \\ \hat{y}_n \end{bmatrix} = \begin{bmatrix} x_1 & 1 \\ x_2 & 1 \\ \vdots & \vdots \\ x_n & 1  \end{bmatrix} \begin{bmatrix} a \\ b \end{bmatrix}
$$

---

Porém, talvez queiramos usar uma função que não seja um polinômio de primeiro grau, como $f(x)=ax+b$. Talvez queiramos, por exemplo, modelar alguma função polinomial. Nesse caso, vamos ter que usar matrizes de dados com mais colunas. Se usarmos a forma: $f(x) = ax^2 + bx + c$, teremos:

$$
 \begin{bmatrix} \hat{y}_1 \\ \hat{y}_2 \\ \vdots \\ \hat{y}_n \end{bmatrix} = \begin{bmatrix} x^2_1 & x_1 & 1 \\ x^2_2 & x_2 & 1 \\ \vdots & \vdots \\ x^2_n & x_n & 1  \end{bmatrix} \begin{bmatrix} a \\ b \\ c \end{bmatrix}
$$

Veja como nosso modelo nos permite calcular o valor dessa função em $x_k=3$ usando:

$$
f(3) = \begin{bmatrix} 3^2 & 3 & 1 \end{bmatrix} \begin{bmatrix} a \\ b \\ c \end{bmatrix} = \begin{bmatrix} 9 & 3 & 1 \end{bmatrix} \begin{bmatrix} a \\ b \\ c \end{bmatrix}
$$

---

No código abaixo, construa as matrizes:

$$
X = \begin{bmatrix} x^2_1 & x_1 & 1 \\ x^2_2 & x_2 & 1 \\ \vdots & \vdots \\ x^2_n & x_n & 1  \end{bmatrix}
$$

e 

$$
y_{\text{medido}} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}
$$

usando os dados gerados (`x` e `y_medido`).

In [24]:
# Dica: use o np.hstack
v = np.array([ [1], [2]])
w = np.array([[ 3], [4]])
z = np.hstack ( (v, 2*w))
print(z)

[[1 6]
 [2 8]]


In [25]:
# Gerando os dados
n = 1000
a, b, c = 1, 2, 3
x = np.linspace(-1, 1, n)
y_real = a * x**2 + b * x + c
sigma_ruido = 0.2
y_medido = y_real + sigma_ruido * np.random.normal(size=y_real.shape)

# Resolva o exercício aqui
X = np.array(x) 
y = np.array(x)

# Testes - deveriam passar quando você acertar o exercício
assert X.shape == (n,3)
assert y.shape == (n,1)

AssertionError: 

# Exercício 3
**Objetivo: calcular parâmetros do modelo usando a pseudoinversa**

Nesse momento, temos uma formulação como:

$$
 \begin{bmatrix} \hat{y}_1 \\ \hat{y}_2 \\ \vdots \\ \hat{y}_n \end{bmatrix} = \begin{bmatrix} x^2_1 & x_1 & 1 \\ x^2_2 & x_2 & 1 \\ \vdots & \vdots \\ x^2_n & x_n & 1  \end{bmatrix} \begin{bmatrix} a \\ b \\ c \end{bmatrix}
$$

Vamos nos referir a essas matrizes usando a notação típica de Machine Learning:

$$
\hat{y} = X w^T
$$

Vamos analisar rapidamente as matrizes que estamos construindo.

A matriz de estimativas $\hat{y}=\begin{bmatrix} \hat{y}_1 \\ \hat{y}_2 \\ \vdots \\ \hat{y}_n \end{bmatrix}$. Ela é uma matriz $n \times 1$, isto é, há $1$ resultado para cada uma das medidas que fizermos.

A matriz de entradas $X=\begin{bmatrix} x^2_1 & x_1 & 1 \\ x^2_2 & x_2 & 1 \\ \vdots & \vdots \\ x^2_n & x_n & 1  \end{bmatrix}$. Ela é uma matriz $n \times k$, onde $n$ é o número de medidas coletadas e $k$ é o número de pequenas funções que usamos para encontrar as nossas estimativas.

Por fim, a matriz $w^T = \begin{bmatrix} a \\ b \\ c \end{bmatrix}$ contém os pesos que serão aplicados a cada um dos resultados colocados na matriz $X$ de forma a gerar as saídas correspondentes.

Nosso problema de estimativa de parâmetros pode ser refraseado da seguinte forma: gostaríamos de estimar os elementos de $w^T$ de tal forma que a estimativa $\hat{y}$ se aproxime dos valores medidos $y$ minimizando o erro calculado por $e^Te$.

---

Nosso modelo gera estimativas para $y$ à partir de medições:

$$
\hat{y} = X w^T
$$

Se o modelo fosse perfeito, ele iria gerar exatamente os valores medidos para $y$:

$$
y = X w^T
$$

sabemos que, se $X$ fosse inversível, encontrar $w^T$ à partir das medições seria muito simples. Porém, $X$ não é sequer uma matriz quadrada.

Vamos usar então a seguinte estratégia. Vamos multiplicar os dois lados da equação, à esquerda, por $X^T$:

$$
X^T y = X^T X w^T
$$

A matriz $ X^T X$ é quadrada e inversível. Então, podemos multiplicar os dois lados da equação por sua inversa, chegando a:

$$
(X^T X)^{-1} X^T y = w^T
$$

A matriz $(X^T X)^{-1} X^T$ é chamada de pseudo-inversa de $X$ e usamos a notação $X^+$ para nos referir a ela.

Veja como agora podemos encontrar $w^T$ em função de nossas medições:

$$
w^T = X^+ y
$$

**Exercício**: Use a pseudo-inversa para calcular $w^T$ usando os dados do Exercício 2.

In [ ]:
# Resolva seu exercício aqui


# Exercício 4
**Objetivo: usar a pseudo-inversa para encontrar parâmetros de uma função linear nos parâmetros**

O código abaixo gera valores para medidas ruidosas de uma função $f(x)$. Implemente a função `recuperar_wt` que recebe os valores de $x$ e os valores medidos de $f(x)$ e retorna os valores de $w^T$.

Antes de começar a fazer seu código, escreva quais são as matrizes, fórmulas e equações que você está implementando, incluindo o que significa cada linha e cada coluna das matrizes que vai usar. Sua notação matemática deve ser tal que ela evidencia qual deveria ser o resultado da função `recuperar_wt`, de forma que você consiga verificar, por conta própria, se o resultado está correto.

In [ ]:
x = np.linspace(-10,10,100)
y_real = np.cos(x) + 1.5 * x**2 + 3 * np.cos(2*x) + 5
sigma_ruido = 0.1
y_medido = y_real + sigma_ruido * np.random.normal(size=y_real.shape)

def recuperar_wt(x, y_medido):
    return np.zeros_like(x)



# Exercício 5
**Aplicar a pseudo-inversa num caso real**

Usando os dados que você coletou sobre o tempo de queda de pequenos objetos em sala, use uma formulação matricial e a pseudo-inversa para encontrar o valor da aceleração da gravidade na sua sala de aula.